Copyright (C) 2022  Kamil Erguler <k.erguler@cyi.ac.cy>
 
This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details (<https://www.gnu.org/licenses/>). 

In [ ]:
using sPop2
using Plots
using Statistics
using Distributions

In [8]:
N = 10.0
mu = 20.0
sd = 5.0

xr = 0:50

function get_init_pop(pop::population{T,H}) where {T<:pop_data_det,H}
    pop.data.n_current[1]
end

function get_init_pop(pop::population{T,H,F}) where {T<:pop_data_det,H,F}
    pop.data.devtable_current[0.0]
end

function simStoch(init::population{T,H,F}) where {T,H,F<:stochastic_update}
    rets = []
    for r in 1:1000
        pop = deepcopy(init)
        ret = [get_init_pop(pop)]
        for n in 1:50
            sz, dev, dead = step_pop(pop, mu, sd, 0.0)
            push!(ret, sz)
        end
        if rets == []
            rets = ret
        else
            rets = cat(rets,ret,dims=2)
        end
    end
    res = []
    for r in 1:size(rets)[1]
        p = quantile(rets[r,:],[0.05,0.5,0.95])
        if res == []
            res = p
        else
            res = hcat(res, p)
        end
    end
    return hcat(0:50,transpose(res))
end


function simDet(init::population{T,H,F}) where {T,H,F<:deterministic_update}
    pop = deepcopy(init)
    retd = [0 get_init_pop(pop)]
    for n in 1:50
        sz, dev, dead = step_pop(pop, mu, sd, 0.0)
        retd = vcat(retd, [n sz])
    end
    return retd
end

# age gamma
sto = population(age_data_sto(), age_gamma(), stochastic_update())
det = population(age_data_det(), age_gamma(), deterministic_update())
add_pop(sto,N,0)
add_pop(det,N,0)
out_sto = simStoch(sto)
out_det = simDet(det)
k, theta = sPop2.age_gamma_pars(mu, sd)

plot(out_sto[:,1], out_sto[:,4], fillrange = out_sto[:,2], c="#9ecae1", label="Stoch. (90% range)")
plot!(out_sto[:,1], out_sto[:,3], lw=4, c="#3182bd", label="Stoch. (median)")
plot!(out_det[:,1], out_det[:,2], line = :scatter, c="black", ms=8, label="Deterministic")
plot!(xr, N*(1.0 .- cdf(Gamma(k,theta),xr)), line= :scatter, c="gray", ms=6, label="Expected")
savefig(string("figures/distribution_AGE_GAMMA.pdf"))

# age nbinom
sto = population(age_data_sto(), age_nbinom(), stochastic_update())
det = population(age_data_det(), age_nbinom(), deterministic_update())
add_pop(sto,N,0)
add_pop(det,N,0)
out_sto = simStoch(sto)
out_det = simDet(det)
k, theta = sPop2.age_nbinom_pars(mu, sd)

plot(out_sto[:,1], out_sto[:,4], fillrange = out_sto[:,2], c="#9ecae1", label="Stoch. (90% range)")
plot!(out_sto[:,1], out_sto[:,3], lw=4, c="#3182bd", label="Stoch. (median)")
plot!(out_det[:,1], out_det[:,2], line = :scatter, c="black", ms=8, label="Deterministic")
plot!(xr, N*(1.0 .- cdf(NegativeBinomial(k,theta),xr .- 1)), line= :scatter, c="gray", ms=6, label="Expected")
savefig(string("figures/distribution_AGE_NBINOM.pdf"))

# age fixed
det = population(age_data_det(), age_fixed(), deterministic_update())
add_pop(det,N,0)
out_det = simDet(det)
plot(out_det[:,1], out_det[:,2], line = :scatter, c="black", ms=8, label="Deterministic")
savefig(string("figures/distribution_AGE_FIXED.pdf"))

# acc fixed
det = population(acc_data_det(), acc_fixed(), deterministic_update())
add_pop(det,N,0)
out_det = simDet(det)
plot(out_det[:,1], out_det[:,2], line = :scatter, c="black", ms=8, label="Deterministic")
savefig(string("figures/distribution_ACC_FIXED.pdf"))

# acc erlang
sto = population(acc_data_sto(), acc_erlang(), stochastic_update())
det = population(acc_data_det(), acc_erlang(), deterministic_update())

add_pop(sto,N,0)
add_pop(det,N,0)
out_sto = simStoch(sto)
out_det = simDet(det)
k, theta = sPop2.acc_erlang_pars(mu, sd)

plot(out_sto[:,1], out_sto[:,4], fillrange = out_sto[:,2], c="#9ecae1", label="Stoch. (90% range)")
plot!(out_sto[:,1], out_sto[:,3], lw=4, c="#3182bd", label="Stoch. (median)")
plot!(out_det[:,1], out_det[:,2], line = :scatter, c="black", ms=8, label="Deterministic")
plot!(xr, N*(1.0 .- cdf(Gamma(k,theta),xr)), line= :scatter, c="gray", ms=6, label="Expected")
savefig(string("figures/distribution_ACC_ERLANG.pdf"))

# acc pascal
sto = population(acc_data_sto(), acc_pascal(), stochastic_update())
det = population(acc_data_det(), acc_pascal(), deterministic_update())
add_pop(sto,N,0)
add_pop(det,N,0)
out_sto = simStoch(sto)
out_det = simDet(det)
k, theta = sPop2.acc_pascal_pars(mu, sd)

plot(out_sto[:,1], out_sto[:,4], fillrange = out_sto[:,2], c="#9ecae1", label="Stoch. (90% range)")
plot!(out_sto[:,1], out_sto[:,3], lw=4, c="#3182bd", label="Stoch. (median)")
plot!(out_det[:,1], out_det[:,2], line = :scatter, c="black", ms=8, label="Deterministic")
plot!(xr, N*(1.0 .- cdf(NegativeBinomial(k,theta),xr .- 1)), line= :scatter, c="gray", ms=6, label="Expected")
savefig(string("figures/distribution_ACC_PASCAL.pdf"))

In [10]:
function get_dev_table(devtable)
    rd = Dict{Float64, Float64}()
    for (x,n) in devtable
        if haskey(rd, x)
            rd[x] += n
        else
            rd[x] = n
        end
    end
    return rd
end

mu = [40.0 20.0]
sd = [5.0 5.0]
theta = sd .* sd ./ mu
k = mu ./ theta

xri = collect(0:199)
xr = collect(0:0.01:50)

a = population(acc_data_det(), acc_erlang(), deterministic_update())
add_pop(a, 100.0, 0.0)

ii = [0 1]
retA = [0 get_pop(a)]
retL = [0 get_dev_table(a.data.devtable_current)]
for n in xri[2:end]
    i = n<=20 ? 1 : 2
    step_pop(a, mu[i], sd[i], 0.0)
    retA = vcat(retA, [n get_pop(a)])
    retL = vcat(retL, [n get_dev_table(a.data.devtable_current)])
    ii = vcat(ii,[n i])
end


plot(xri,retA[:,2], c="black", lw=2, legend=false)
for i in [2 1]
    plot!(xr,100*(1.0 .- cdf(Gamma(k[i],theta[i]), xr)), c="gray", lw=3, ls=:dash, legend=false)
end
xlims!((0, 50))
savefig(string("figures/switch.pdf"))

plot(retL[1,2], c="black", legend = false)
for r in 2:50
    plot!(retL[r,2], c="black", lw=retL[r,1] in [5 20 29] ? 3 : 1, legend = false)
end
ylims!((0,20))
savefig("figures/switch_devc.pdf")

LoadError: MethodError: no method matching get_pop(::Dict{Float64, Float64})
[0mClosest candidates are:
[0m  get_pop([91m::population{T, H, F}[39m) where {T<:age_data, H, F} at ~/Desktop/git/sPop2.jl/src/sPop2.jl:274
[0m  get_pop([91m::population{T, H, F}[39m) where {T<:acc_data, H, F} at ~/Desktop/git/sPop2.jl/src/sPop2.jl:278